In [ ]:
# Standard Library Imports
from itertools import combinations
from collections import defaultdict

# Scientific Computing
import numpy as np
from sympy import symbols

# PyCalphad (Thermodynamics Calculations & Plotting)
from pycalphad import (
    Database, calculate, equilibrium, variables as v
)

# Suppress NumPy warnings
np.seterr(all='ignore')

# Custom
from symbolic_hulls import *


In [ ]:
# Load database (ensure path is correct)
db = Database(r'../TDDatabaseFiles_temp/alfe.tdb')

# Extract available phases and elements
phases = list(db.phases.keys())  
constituents = list(db.elements)

print("Phases in Database:", phases)
print("Elements in Database:", constituents)

# Define components, ensure 'VA' is included
comps = ['AL', 'FE', 'VA']

for t in np.linspace(300, 2000, 100):
    conditions = { 
                    v.T: t,
                    v.P: 101325,
                    v.X("AL"): np.linspace(0, 1, 100)
                }

    try:
        # Define equilibrium conditions properly
        eq = equilibrium(db, comps, phases, conditions=conditions)
        print(f"Equilibrium calculated at T={t} K")
    except Exception as e:
        print(f"Error at T={t} K: {e}")
        continue

In [ ]:
# Load in polynomials dictionary
phase_poly_dict = load_sympy_dict_from_json('binary_polynomials.json')

# Calculate the equilibrium phases between all pairs of phases using the fitted polynomials
pairs = list(combinations(phase_poly_dict.keys(), 2))
equilibrium_eqs_dict = defaultdict(list)
for pair in pairs:
    phase1 = pair[0]
    phase2 = pair[1]

    proj, variables, pvariables = projection_function(phase_poly_dict[phase1], phase_poly_dict[phase2])
    proj = proj.replace(lambda term: term.is_Number, lambda term: int(round(term, 0)))   # We are going to round the projection funtion
    discriminant = recursive_discriminant(proj, variables)
    solutions = sp.solve(discriminant, symbols('s_p'))
    equilibrium_eqs_dict[phase1].extend(solutions)

    proj, variables, pvariables = projection_function(phase_poly_dict[phase2], phase_poly_dict[phase1])
    proj = proj.replace(lambda term: term.is_Number, lambda term: int(round(term, 0)))   # We are going to round the projection funtion
    discriminant = recursive_discriminant(proj, variables)
    solutions = sp.solve(discriminant, symbols('s_p'))
    equilibrium_eqs_dict[phase2].extend(solutions)

# Calculate the equilibrium points for each composition
points = np.empty((0, 3))
x_vals = np.linspace(0, 1, 100)
for phase in phase_poly_dict.keys():
    print(phase)
    phase_func = sp.lambdify(symbols('x s'), phase_poly_dict[phase], 'numpy')
    for eq_func in equilibrium_eqs_dict[phase]:
        eq_func = sp.lambdify(symbols('x_p'), eq_func, 'numpy')
        s_vals = eq_func(x_vals)
        energy_vals = phase_func(x_vals, s_vals)
        points = np.concatenate((points, np.column_stack((x_vals, s_vals, energy_vals))), axis=0)

points = points[~np.isnan(points).any(axis=1)]

# Run the lower hull
lower_convex_hull(points)